<a href="https://colab.research.google.com/github/ducleins/eurusd_2026/blob/main/02_Notebook/02_Clean_Sheet1_Dealer_CTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# CELL 1: SETUP VÀ KHAI BÁO BIẾN
# =========================================================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# 1. Kết nối Google Drive (Chỉ chạy nếu Colab chưa kết nối)
# drive.mount('/content/drive')

# 2. Định nghĩa các đường dẫn chính
BASE_PATH = '/content/drive/MyDrive/EURUSD_2026'
RAW_CFTC_PATH = os.path.join(BASE_PATH, '01_Raw', 'cftc')
FILTERED_PATH = os.path.join(BASE_PATH, '01_Raw', 'filtered')

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(FILTERED_PATH, exist_ok=True)

# 3. Ánh xạ tên cột gốc sang tên chuẩn (Dựa trên 4 file thành công)
COLUMN_MAPPING = {
    'Report_Date_as_YYYY-MM-DD': 'Date',
    'Open_Interest_All': 'Open_Interest_All',
    'Dealer_Positions_Long_All': 'Dealer_Long_All',
    'Dealer_Positions_Short_All': 'Dealer_Short_All',
    'Lev_Money_Positions_Long_All': 'CTA_Long_All',
    'Lev_Money_Positions_Short_All': 'CTA_Short_All',
    'Market_and_Exchange_Names': 'Market_and_Exchange_Names' # Cột quan trọng để lọc
}

In [2]:
# =========================================================
# CELL 2: HÀM XỬ LÝ DỮ LIỆU VÀ TÍNH TOÁN CHỈ SỐ CƠ BẢN
# =========================================================

def process_cftc_file(file_path):
    """
    Chức năng: Đọc file CFTC, lọc theo 'EURO FX', tính Dealer/CTA Net/Pct.
    Xử lý linh hoạt cho file có/không có header.
    """

    # 1. THỬ ĐỌC FILE (Thử với header trước, delimiter là ',')
    try:
        df = pd.read_csv(file_path, sep=',', encoding='utf-8')
        # Kiểm tra sự tồn tại của cột lọc. Nếu không có, có thể header bị lỗi.
        if COLUMN_MAPPING['Market_and_Exchange_Names'] not in df.columns:
            raise ValueError("Lỗi tiêu đề.")
        is_header_present = True

    except (KeyError, ValueError, pd.errors.ParserError):
        # 1b. THỬ ĐỌC LẠI MÀ KHÔNG CÓ HEADER
        df = pd.read_csv(file_path, sep=',', encoding='utf-8', header=None)
        is_header_present = False

        # Gán tên cột theo Index cho các cột cần thiết (Dựa trên vị trí đã kiểm tra)
        COL_INDICES_FOR_FILTER = {
            0: 'Market_and_Exchange_Names', # Cột 0: Tên Hàng Hóa/Tiền Tệ
        }
        df.rename(columns=COL_INDICES_FOR_FILTER, inplace=True)

    # 2. LỌC DỮ LIỆU CỦA EURUSD
    # Lọc theo mã chuẩn 'EURO FX'
    df_euro = df[df['Market_and_Exchange_Names'].str.contains('EURO FX', na=False, case=False)].copy()

    if df_euro.empty:
        raise ValueError("Không tìm thấy dữ liệu 'EURO FX' sau khi lọc.")

    # 3. CHUẨN HÓA CỘT VÀ ÁNH XẠ

    # Nếu không có header, phải gán tên cột cho các chỉ số vị thế theo Index
    if not is_header_present:
        COL_INDICES_FULL = {
            # 0 đã gán, 2 là Date
            2: 'Report_Date_as_YYYY-MM-DD',
            7: 'Open_Interest_All',
            8: 'Dealer_Positions_Long_All',
            9: 'Dealer_Positions_Short_All',
            14: 'Lev_Money_Positions_Long_All',
            15: 'Lev_Money_Positions_Short_All',
        }
        df_euro.rename(columns=COL_INDICES_FULL, inplace=True)

    # Lấy các cột cần thiết và đổi tên
    df_clean = df_euro[list(COLUMN_MAPPING.keys())].rename(columns=COLUMN_MAPPING)

    # 4. TÍNH TOÁN CHỈ SỐ CƠ BẢN
    df_clean['Date'] = pd.to_datetime(df_clean['Date'])

    # Dealer Net
    df_clean['Dealer_Net'] = df_clean['Dealer_Long_All'] - df_clean['Dealer_Short_All']
    df_clean['Dealer_%OI'] = (df_clean['Dealer_Net'] / df_clean['Open_Interest_All']) * 100

    # CTA Net
    df_clean['CTA_Net'] = df_clean['CTA_Long_All'] - df_clean['CTA_Short_All']
    df_clean['CTA_%OI'] = (df_clean['CTA_Net'] / df_clean['Open_Interest_All']) * 100

    # Giữ lại các cột cần xuất
    FINAL_COLS = ['Date', 'Open_Interest_All', 'Dealer_Net', 'Dealer_%OI', 'CTA_Net', 'CTA_%OI']

    return df_clean[FINAL_COLS]


# =========================================================
# CELL 3: THỰC THI GỘP TẤT CẢ FILE VÀ LƯU
# =========================================================

def combine_and_save_cftc():
    """Chức năng: Quét, xử lý, gộp tất cả file CFTC và lưu ra file filtered."""

    FILE_NAMES = [f for f in os.listdir(RAW_CFTC_PATH) if f.endswith('.txt') or f.endswith('.csv')]
    all_data = []

    print(f"--- BẮT ĐẦU XỬ LÝ {len(FILE_NAMES)} FILE CFTC RAW ---")

    for filename in FILE_NAMES:
        file_path = os.path.join(RAW_CFTC_PATH, filename)

        try:
            df_processed = process_cftc_file(file_path)
            all_data.append(df_processed)
            print(f"✅ Xử lý thành công: {filename}")

        except Exception as e:
            # Nếu có lỗi, chỉ in ra cảnh báo và bỏ qua file lỗi đó
            print(f"⚠️ BỎ QUA file lỗi {filename}: {e}")
            continue

    # Gộp dữ liệu và làm sạch
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df = combined_df.sort_values('Date').drop_duplicates(subset=['Date'])

        # Lưu ra file cuối cùng
        output_file = os.path.join(FILTERED_PATH, 'cftc_dealer_cta_filtered.csv')
        combined_df.to_csv(output_file, index=False)

        print(f"\n--- HOÀN THÀNH XỬ LÝ CFTC ---")
        print(f"Lưu file tại: {output_file}")
        print(f"Kích thước dữ liệu cuối cùng: {combined_df.shape}")

    else:
        print("\n❌ LỖI NGHIÊM TRỌNG: Không có file nào được xử lý thành công.")

# Chạy hàm chính
combine_and_save_cftc()

--- BẮT ĐẦU XỬ LÝ 5 FILE CFTC RAW ---
✅ Xử lý thành công: FinFut_2022.txt
✅ Xử lý thành công: FinFut_2023.txt
✅ Xử lý thành công: FinFut_2024.txt
✅ Xử lý thành công: FinFut_2025.txt
✅ Xử lý thành công: FinFut_21102025.txt

--- HOÀN THÀNH XỬ LÝ CFTC ---
Lưu file tại: /content/drive/MyDrive/EURUSD_2026/01_Raw/filtered/cftc_dealer_cta_filtered.csv
Kích thước dữ liệu cuối cùng: (199, 6)
